<a href="https://colab.research.google.com/github/KJ-black/Machine-Learning-on-Material-Informatics/blob/main/projectile_motion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Generation

In [9]:
import numpy as np
import random

n = 100000 # number of dataset
data = []
for i in range(n):
  angle = random.uniform(0, np.pi/2)
  v = random.uniform(0, 100)
  h = ((v*np.sin(angle))**2)/(2*9.8)
  d = (v**2*2*np.sin(angle)*np.cos(angle))/9.8
  data.append([angle, v, h, d])
data = np.array(data)